# Build Multimodal RAG with Amazon OpenSearch Service

In this notebook, you will build and run multimodal search using a sample retail dataset. You will use multimodal generated embeddings for text and image and experiment by running text search only, image search only and both text and image search in OpenSearch Service.

You will be using a retail dataset that contains 2,465 retail product samples that belong to different categories such as accessories, home decor, apparel, housewares, books, and instruments. Each product contains metadata including the ID, current stock, name, category, style, description, price, image URL, and gender affinity of the product. You will be using only the product image and product description fields in the solution.

 
Step 1: Create embeddings for text and images

Step 2: Store the embeddings in OpenSearch Service index

Step 3: Use LLM to generate text using the context from OpenSearch




---

Step 1: 

1. Build AI/connector between AOS and Embedding model - Titan Mulitmodal embeddings model
2. Register/Deploy the Embedding model in AOS
3. Create a KNN index in AOS
4. Create an ingest pipeline to generate the embedding inside AOS

Step 2:

1. Index the data

Step 3:

1. Run multimodal neural search query in AOS 
2. Feed the LLM with the extract results from AOS - Claude Sonnet 3 
2.1. Build AI/connector between AOS and LLM to generate the text
2.2. Register/Deploy the LLM in AOS
2.3. Run conversational search query in AOS (using the LLM model)

## 1. Lab Pre-requisites


For this notebook we require a few libraries. We'll use the Python clients for Amazon OpenSearch Service and Amazon Bedrock, and OpenSearch ML Client library for generating multimodal embeddings.

#### 1.1. Import libraries & initialize resource information
The line below will import all the relevant libraries and modules used in this notebook.

In [ ]:
import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
from sagemaker import get_execution_role
import random 
import string
import s3fs
from urllib.parse import urlparse
from IPython.display import display, HTML
from alive_progress import alive_bar
from opensearch_py_ml.ml_commons import MLCommonClient
from requests_aws4auth import AWS4Auth
import requests 

#### 1.2. Get CloudFormation stack output variables

We have preconfigured a few resources by creating a CloudFormation stack in the account. Names and ARN of these resources will be used within this lab. We are going to load some of the information variables here.

In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "advaned-rag-opensearch"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

## 2. Prepare the dataset

### 2.1.Download the dataset (.gz) and extract the .gz file

In [ ]:
import os
import urllib.request
import tarfile

os.makedirs('tmp/images', exist_ok = True)
metadata_file = urllib.request.urlretrieve('https://aws-blogs-artifacts-public.s3.amazonaws.com/BDB-3144/products-data.yml', 'tmp/images/products.yaml')
img_filename,headers= urllib.request.urlretrieve('https://aws-blogs-artifacts-public.s3.amazonaws.com/BDB-3144/images.tar.gz', 'tmp/images/images.tar.gz')              
print(img_filename)
file = tarfile.open('tmp/images/images.tar.gz')
file.extractall('tmp/images/')
file.close()
#remove images.tar.gz
os.remove('tmp/images/images.tar.gz')

## 3. Create a connection with OpenSearch domain.
Next, we'll use Python API to set up connection with OpenSearch domain.

#### Important pre-requisite
You should have followed the steps in the Lab instruction section to map Sagemaker notebook role to OpenSearch `ml_full_access` role. If not, please visit the lab instructions and complete the **Setting up permission for Notebook IAM Role** section.

#### Retrieving credentials from Secrets manager
We are going to use Amazon Sagemaker Notebook IAM role to configure the workflows in OpenSearch. This IAM Role has permission to pass BedrockInference IAM role to OpenSearch. OpenSearch will then be able to use BedrockInference IAM role to make calls to Bedrock models.

##### NOTE: 
_At any point in this lab, if you get a failure message - **The security token included in the request is expired.**_ You can resolve it by running this cell again. The cell refreshes the security credentials that is required for the rest of the lab.

In [ ]:
kms = boto3.client('secretsmanager')
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['OpenSearchSecret'])['SecretString'])

#credentials = boto3.Session().get_credentials()
#auth = AWSV4SignerAuth(credentials, region)
auth = (aos_credentials['username'], aos_credentials['password'])

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
ml_client = MLCommonClient(aos_client)

#initializing some variables that we will use later.

embedding_connector_id = ""
embedding_model_id = ""

## 4. Create and deploy model connector to Amazon Bedrock Titan Multimodal Embedding

Following cell will create a connector using SageMaker Notebook IAM role. Following cell will create a OpenSearch remote model connector with Amazon Bedrock Titan MM Embedding model. Following cell defines the connector configuration.

## 3. Create the OpenSearch Bedrock ML connector

you need to change **"iam-role-arn"** below with the ARN of the IAM role that has permissions to talk to OpenSearch and mapped as back-end role in OpenSearch dashboards

In [ ]:
import boto3
import requests 
from requests_aws4auth import AWS4Auth
import json


if not embedding_connector_id:
    host = f'https://{aos_host}/'
    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


    # Register repository
    path = '_plugins/_ml/connectors/_create'
    url = host + path

    payload = {
        "name": "Amazon Bedrock Connector: embedding",
        "description": "The connector to bedrock Titan multimodal embedding model",
        "version": 1,
        "protocol": "aws_sigv4",
        "credential": {
          "roleArn": f"arn:aws:iam::{account_id}:role/{bedrock_inf_iam_role}"
       },
       "parameters": {
        "region": region,
        "service_name": "bedrock",
        "model": "amazon.titan-embed-image-v1"
       },
       "actions": [
        {
          "action_type": "predict",
          "method": "POST",
          "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
          "headers": {
            "content-type": "application/json",
            "x-amz-content-sha256": "required"
          },
         "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
         "pre_process_function": "connector.pre_process.bedrock.embedding",
         "post_process_function": "connector.post_process.bedrock.embedding"}
       ]
    }
    headers = {"Content-Type": "application/json"}

    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    print(r.status_code)
    print(r.text)
    embedding_connector_id = json.loads(r.text)["connector_id"]
else:
    print(f"Connector already exists - {embedding_connector_id}")
    
embedding_connector_id

Once the model connector is defined. We need to register the model and deploy. Following two cells will register and then deploy the model connection respectively.

Once the model connector is defined. We need to register the model and deploy. Following two cells will register and then deploy the model connection respectively.

In [ ]:
# Register the embedding model
if not embedding_model_id:
    path = '_plugins/_ml/models/_register'
    url = 'https://'+aos_host + '/' + path
    payload = { "name": "Bedrock Titan mm embeddings model",
    "function_name": "remote",
    "description": "Bedrock Titan mm embeddings model",
    "connector_id": embedding_connector_id}
    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    embedding_model_id = json.loads(r.text)["model_id"]
else:
    print("skipping model registration - model already exists")
print("Model registered under model_id: "+embedding_model_id)

In [ ]:
# Deploy the embedding model
path = '_plugins/_ml/models/'+embedding_model_id+'/_deploy'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, headers=headers)
deploy_status = json.loads(r.text)["status"]
print("Deployment status of the model, "+embedding_model_id+" : "+deploy_status)

## 4. Test the OpenSearch - Bedrock integration with a test input

In [ ]:

import base64


path = '_plugins/_ml/models/'+embedding_model_id+'/_predict'
url = 'https://'+aos_host + '/' + path
img = "tmp/images/footwear/2d2d8ec8-4806-42a7-b8ba-ceb15c1c7e84.jpg"
with open(img, "rb") as image_file:
    input_image_binary = base64.b64encode(image_file.read()).decode("utf8")
    
payload = {
"parameters": {
"inputText": "Sleek, stylish black sneakers made for urban exploration. With fashionable looks and comfortable design, these sneakers keep your feet looking great while you walk the city streets in style",
"inputImage":input_image_binary
}
}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)

try:
    embed = json.loads(r.text)['inference_results'][0]['output'][0]['data'][0:10]
    shape = json.loads(r.text)['inference_results'][0]['output'][0]['shape'][0]
    print("First 10 dimensions:")
    print(str(embed))
    print("\n")
    print("Total: " + str(shape) + " dimensions")
except KeyError as e:
    print(f"KeyError: {e}")
    print("The response does not contain the expected data structure.")
except Exception as e:
    print(f"Error: {e}")
    print("An unexpected error occurred.")

In [ ]:
print(r.text)

In [ ]:
print(str(path))

## 5. Create the OpenSearch ingest pipeline

## 5. Create ingest pipeline
Let's create an ingestion pipeline that will call Amazon Bedrock Titan Multimodal embedding model and convert the text and image into multimodal vector embedding. Ingest pipeline is a feature in OpenSearch that allows you to define certain actions to be performed at the time of data ingestion. You could do simple processing such as adding a static field, modify an existing field, or call a remote model to get inference and store inference output together with the indexed record/document. In our case inference output is vector embedding.

Following ingestion pipeline is going to call our remote model and convert product image `product_description` field and the `image_binary` to vector and store it in the field called `vector_embedding`

In [ ]:
path = "_ingest/pipeline/bedrock-multimodal-ingest-pipeline"
url = 'https://'+aos_host + '/' + path
payload = {
"description": "A text/image embedding pipeline",
"processors": [
{
"text_image_embedding": {
"model_id":embedding_model_id,
"embedding": "vector_embedding",
"field_map": {
"text": "product_description",
"image": "image_binary"
}}}]}
r = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)

## 6. Create the k-NN index

In [ ]:
path = "bedrock-multimodal-rag"
url = 'https://'+aos_host + '/' + path

#this will delete the index if already exists
requests.delete(url, auth=awsauth, json=payload, headers=headers)

payload = {
  "settings": {
    "index.knn": True,
    "default_pipeline": "bedrock-multimodal-ingest-pipeline"
  },
  "mappings": {
      
    "_source": {
     
      "excludes": ["image_binary"]
    },
    "properties": {
      "vector_embedding": {
        "type": "knn_vector",
        "dimension": shape,
        "method": {
          "name": "hnsw",
          "engine": "faiss",
          "parameters": {}
        }
      },
      "product_description": {
        "type": "text"
      },
        "image_url": {
        "type": "text"
      },
      "image_binary": {
        "type": "binary"
      }
    }
  }
}
r = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)

## 7. Ingest the dataset into k-NN index usig Bulk request

In [ ]:
from ruamel.yaml import YAML
from PIL import Image
import os
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

def resize_image(photo, width, height):
    Image.MAX_IMAGE_PIXELS = 100000000
    
    with Image.open(photo) as image:
        image.verify()
    with Image.open(photo) as image:    
        
        if image.format in ["JPEG", "PNG"]:
            file_type = image.format.lower()
            path = image.filename.rsplit(".", 1)[0]

            image.thumbnail((width, height))
            image.save(f"{path}-resized.{file_type}")
    return file_type, path

# Load the products from the dataset
yaml = YAML()
items_ = yaml.load(open('tmp/images/products.yaml'))

batch = 0
count = 0
body_ = ''
batch_size = 100
last_batch = int(len(items_)/batch_size)
action = json.dumps({ 'index': { '_index': 'bedrock-multimodal-rag' } })

for item in items_:
    count+=1
    fileshort = "tmp/images/"+item["category"]+"/"+item["image"]
    payload = {}
    payload['image_url'] = fileshort
    payload['product_description'] = item['description']
    
    #resize the image and generate image binary
    file_type, path = resize_image(fileshort, 2048, 2048)

    with open(fileshort.split(".")[0]+"-resized."+file_type, "rb") as image_file:
        input_image = base64.b64encode(image_file.read()).decode("utf8")
    
    os.remove(fileshort.split(".")[0]+"-resized."+file_type)
    payload['image_binary'] = input_image
    
    body_ = body_ + action + "\n" + json.dumps(payload) + "\n"
    
    if(count == batch_size):
        response = aos_client.bulk(
        index = 'bedrock-multimodal-rag',
        body = body_
        )
        batch += 1
        count = 0
        print("batch "+str(batch) + " ingestion done!")
        if(batch != last_batch):
            body_ = ""
        
            
#ingest the remaining rows
response = aos_client.bulk(
        index = 'bedrock-multimodal-rag',
        body = body_
        )
        
print("All "+str(last_batch)+" batches ingested into index")


## 8. Experiment 1: Keyword search

In [ ]:
#Keyword Search
query = "trendy footwear for women"
url = 'https://' + aos_host + "/bedrock-multimodal-rag/_search"
keyword_payload = {"_source": {
        "exclude": [
            "vector_embedding"
        ]
        },
        "query": {    "match": {
                        "product_description": {
                            "query": query
                        }
                        }
                    }
        
        ,"size":5,
  }

r = requests.get(url, auth=awsauth, json=keyword_payload, headers=headers)
response_ = json.loads(r.text)
docs = response_['hits']['hits']

for i,doc in enumerate(docs):
    print(str(i+1)+ ". "+doc["_source"]["product_description"])
    image = Image.open(doc["_source"]["image_url"])
    image.show()

## 9. Experiment 2: Multimodal search with only text caption as input

In [ ]:
#Multimodal Search
#Text as input

query = "trendy footwear for women"
url = 'https://'+aos_host+"/bedrock-multimodal-rag/_search"
text_embedding_payload = {"_source": {
        "exclude": [
            "vector_embedding"
        ]
        },
        "query": {    
       
        "neural": {
            "vector_embedding": {
                
            #"query_image":query_image_binary,
            "query_text":query,
                
            "model_id": embedding_model_id,
            "k": 3
            }
            }
            
                    }
        
        ,"size":5,
  }

r = requests.get(url, auth=awsauth, json=text_embedding_payload, headers=headers)
response_ = json.loads(r.text)
docs = response_['hits']['hits']

for i,doc in enumerate(docs):
    print(doc["_source"]["product_description"])
    image = Image.open(doc["_source"]["image_url"])
    image.show()
   
    

## 10. Experiment 3: Multimodal search with only image as input

In [ ]:
#Multimodal Search
#image as input
import urllib.request

s3 = boto3.client('s3')
image_file = urllib.request.urlretrieve('https://aws-blogs-artifacts-public.s3.amazonaws.com/BDB-3144/women_wear.jpg', 'tmp/women-footwear-1.jpg')
img = Image.open("tmp/women-footwear-1.jpg") 
print("Input query Image:")
img.show()
with open("tmp/women-footwear-1.jpg", "rb") as image_file:
    query_image_binary = base64.b64encode(image_file.read()).decode("utf8")

In [ ]:
print(str(query_image_binary))

In [ ]:
url = 'https://'+aos_host+"/bedrock-multimodal-rag/_search"
keyword_payload = {"_source": {
        },
        "query": {    
       
        "neural": {
            "vector_embedding": {
            "query_image":query_image_binary,
            "model_id": embedding_model_id,
            "k": 5
            }
            
            }
                    }
        
        ,"size":5,
  }

r = requests.get(url, auth=awsauth, json=keyword_payload, headers=headers)
response_ = json.loads(r.text)
docs = response_['hits']['hits']

for i,doc in enumerate(docs):
    print(doc["_source"]["product_description"])
    image = Image.open(doc["_source"]["image_url"])
    image.show()

In [ ]:
print(r.text)

In [ ]:
print(str(headers))

In [ ]:
print(str(img_embedding_payload))

In [ ]:
print(r.text)

## 11. Experiment 4: Multimodal search with both image and text caption as inputs

In [ ]:
#Multimodal Search
#Text and image as inputs
import urllib.request
s3 = boto3.client('s3')
url = 'https://'+aos_host + "/bedrock-multimodal-rag/_search"
query = "trendy footwear for women"
print("Input text query: "+query)
# urllib.request.urlretrieve( 
#   'https://cdn.pixabay.com/photo/2014/09/03/20/15/shoes-434918_1280.jpg',"tmp/women-footwear.jpg") 
img = Image.open("tmp/women-footwear-1.jpg") 
print("Input query Image:")
img.show()
with open("tmp/women-footwear-1.jpg", "rb") as image_file:
    query_image_binary = base64.b64encode(image_file.read()).decode("utf8")
keyword_payload = {"_source": {
        "exclude": [
            "vector_embedding"
        ]
        },
        "query": {    
       
        "neural": {
            "vector_embedding": {
                
            "query_image":query_image_binary,
            "query_text":query,
                
            "model_id": embedding_model_id,
            "k": 5
            }
            
            }
                    }
        
        ,"size":5,
  }

r = requests.get(url, auth=awsauth, json=keyword_payload, headers=headers)
response_ = json.loads(r.text)
docs = response_['hits']['hits']

for i,doc in enumerate(docs):
    print(doc["_source"]["product_description"])
    image = Image.open(doc["_source"]["image_url"])
    image.show()

## 12. Create the OpenSearch Bedrock Claude LLM connector

you need to change **"iam-role-arn"** below with the ARN of the IAM role that has permissions to talk to OpenSearch and mapped as back-end role in OpenSearch dashboards

In [ ]:
import boto3
import requests 
from requests_aws4auth import AWS4Auth
import json

#initializing variables that we will use later.

llm_connector_id = ""
llm_model_id = ""

if not llm_connector_id:
    host = f'https://{aos_host}/'
    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


    # Register repository
    path = '_plugins/_ml/connectors/_create'
    url = host + path

    payload = {
        "name": "Amazon Bedrock Connector: Claude 3 sonnet",
        "description": "The connector to bedrock Claude 3",
        "version": 1,
        "protocol": "aws_sigv4",
        "credential": {
          "roleArn": f"arn:aws:iam::{account_id}:role/{bedrock_inf_iam_role}"
       },
       "parameters": {
        "region": region,
        "service_name": "bedrock",
        "model": "anthropic.claude-3-sonnet-20240229-v1:0",
        "response_filter": "$.content[0].text",
        "max_tokens_to_sample": "8000",
        "anthropic_version": "bedrock-2023-05-31",
       },
       "actions": [
        {
          "action_type": "predict",
          "method": "POST",
          "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
          "headers": {
            "content-type": "application/json",
            "x-amz-content-sha256": "required"
          },
         "request_body": "{\"messages\":[{\"role\":\"user\",\"content\":[{\"type\":\"text\",\"text\":\"${parameters.inputs}\"}]}],\"anthropic_version\":\"${parameters.anthropic_version}\",\"max_tokens\":${parameters.max_tokens_to_sample}}"}
       ]
    }
    headers = {"Content-Type": "application/json"}

    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    print(r.status_code)
    print(r.text)
    llm_connector_id = json.loads(r.text)["connector_id"]
else:
    print(f"Connector already exists - {llm_connector_id}")
    
llm_connector_id

## 13. Register and deploy the OpenSearch Bedrock Claude LLM

In [ ]:
# Register the llm model
if not llm_model_id:
    path = '_plugins/_ml/models/_register'
    url = 'https://'+aos_host + '/' + path
    payload = { "name": "Amazon Bedrock Connector: Claude 3 sonnet",
    "function_name": "remote",
    "description": "The connector to bedrock Claude 3",
    "connector_id": llm_connector_id}
    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    llm_model_id = json.loads(r.text)["model_id"]
else:
    print("skipping model registration - llm model already exists")
print("Model registered under llm_model_id: "+llm_model_id)

In [ ]:
# Deploy the llm model
path = '_plugins/_ml/models/'+llm_model_id+'/_deploy'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, headers=headers)
deploy_status = json.loads(r.text)["status"]
print("Deployment status of the model, "+llm_model_id+" : "+deploy_status)

## 14. Test LLM model inference

In [ ]:
# You can test the llm text generation here.
payload = {
    "parameters":
    { 
        "inputs": "what are the most popular shoes styles for women?"
    } 
}

path = '_plugins/_ml/models/'+llm_model_id+'/_predict'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, json=payload , headers=headers )


#print status of the API call.
print(f"Status: {r.status_code}. Response:{r.text}")

if r.status_code == 200:
    llm_gen = json.loads(r.text)['inference_results'][0]['output'][0]
    print(str(llm_gen))

In [ ]:
#initializing variables that we will use later.

agent_id = ""

## 15. Register and execute an agent

In [ ]:
# you’ll use the embedding model and the Claude model to create a flow agent

payload = {
    "name": "Fashion stylist agent",
  "type": "conversational_flow",
  "description": "this is a demo agent for fashion stylist",
  "app_type": "rag",
     "memory": {
        "type": "conversation_index"
    },
  "tools": [
    {
      "type": "VectorDBTool",
        "name": "shopping_knowledge_base",
      "parameters": {
        "model_id": embedding_model_id,
        "index": "bedrock-multimodal-rag",
        "embedding_field": "vector_embedding",
        "source_field": [
            "product_description"
        ],
          
        "input": "${parameters.question}"
      }
    },
    {
      "type": "MLModelTool",
        "name": "bedrock_claude_v3_sonnet_model",
      "description": "A general tool to answer any question",
      "parameters": {
        "model_id": llm_model_id,
        "prompt": "\n\nHuman:You are a professional fashion stylist. You will always recommend product based on the given context. If you don't have enough context, you will ask Human to provide more information. If you don't see any related product to recommend, just say we don't have such product. If you don't know the answer, just say you don't know. \n\nContext:\n${parameters.shopping_knowledge_base.output:-}\n\n${parameters.chat_history:-}\n\nHuman:${parameters.question}\n\nAssistant:"
      }
    }
  ]
}

path = '_plugins/_ml/agents/_register'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, json=payload , headers=headers )

agent_id = json.loads(r.text)["agent_id"]

#print status of the API call.
print(f"Status: {r.status_code}. Response:{r.text}")


In [ ]:
## Test the agent

In [ ]:
# You can inspect the agent by sending a request to the agents endpoint and providing the agent ID:


# You can test the llm text generation here.
payload = {
    "parameters":
    { #"inputs": "\n\nHuman:hello\n\nAssistant:",
        "inputs": "what are the most trendy shoes for women?",
        "question": "what are the most trendy shoes for women?"
    } 
}

path = '_plugins/_ml/agents/'+agent_id+'/_execute'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, json=payload , headers=headers)

if r.status_code == 200:
    llm_agent_gen = json.loads(r.text)['inference_results'][0]['output'][0]['result'][0]
    print(str(llm_agent_gen))


#print status of the API call.
print(f"Status: {r.status_code}. Response:{r.text}")



# Streamlit application deployment

## Write our code to app.py in the local file system

Run the code block below to write its contents to app.py

In [ ]:
%%writefile ../../app.py

import boto3
import json
# import pandas as pd
from tqdm import tqdm
# import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
# from sagemaker import get_execution_role
# import random 
# import string
# import s3fs
# from urllib.parse import urlparse
# from IPython.display import display # , HTML
from opensearch_py_ml.ml_commons import MLCommonClient
from requests_aws4auth import AWS4Auth
import requests 
from PIL import Image
import streamlit as st

temp_dir = "/home/ec2-user/SageMaker/advanced-rag-amazon-opensearch/retrieval-augment-generation/"

st.set_page_config(layout="wide")
st.title('Vector search with Amazon OpenSearch Service')

# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "multimodal-rag-opensearch"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

## Create a connection with OpenSearch domain.
# Retrieving credentials from Secrets manager¶
kms = boto3.client('secretsmanager')
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['OpenSearchSecret'])['SecretString'])

#credentials = boto3.Session().get_credentials()
#auth = AWSV4SignerAuth(credentials, region)
auth = (aos_credentials['username'], aos_credentials['password'])

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
ml_client = MLCommonClient(aos_client)
#initializing some variables that we will use later.
embedding_connector_id = ""
embedding_model_id = ""

# Create the OpenSearch Bedrock ML connector¶
if not embedding_connector_id:
    host = f'https://{aos_host}/'
    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


    # Register repository
    path = '_plugins/_ml/connectors/_create'
    url = host + path

    payload = {
        "name": "Amazon Bedrock Connector: embedding",
        "description": "The connector to bedrock Titan multimodal embedding model",
        "version": 1,
        "protocol": "aws_sigv4",
        "credential": {
          "roleArn": f"arn:aws:iam::{account_id}:role/{bedrock_inf_iam_role}"
       },
       "parameters": {
        "region": region,
        "service_name": "bedrock",
        "model": "amazon.titan-embed-image-v1"
       },
       "actions": [
        {
          "action_type": "predict",
          "method": "POST",
          "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
          "headers": {
            "content-type": "application/json",
            "x-amz-content-sha256": "required"
          },
         "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
         "pre_process_function": "connector.pre_process.bedrock.embedding",
         "post_process_function": "connector.post_process.bedrock.embedding"}
       ]
    }
    headers = {"Content-Type": "application/json"}

    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    if r.status_code != 200:
        st.write(r.status_code)
    embedding_connector_id = json.loads(r.text)["connector_id"]
else:
    st.write(f"Connector already exists - {embedding_connector_id}")

#tab1, right_column = st.columns(2)

tab1, tab2 = st.tabs(["Keyword Search", "Multimodal Search"])

with tab1:
    st.header("Keyword Search")
    # st.image("https://static.streamlit.io/examples/cat.jpg", width=200)
with tab2:
    st.header("Multimodal Search")
    # st.image("https://static.streamlit.io/examples/dog.jpg", width=200)

# Text search
# tab1.header("Keyword Search")
query_1 = tab1.text_input("Keywords",placeholder="Type your search here.")
url_1 = 'https://' + aos_host + "/bedrock-multimodal-rag/_search"
keyword_payload_1 = {"_source": {
        "exclude": [
            "vector_embedding"
        ]
        },
        "query": {    "match": {
                        "product_description": {
                            "query": query_1
                        }
                        }
                    }
        ,"size":5,
  }

r_1 = requests.get(url_1, auth=awsauth, json=keyword_payload_1, headers=headers)
response_1 = json.loads(r_1.text)
docs_1 = response_1['hits']['hits']

for i,doc in enumerate(docs_1):
    tab1.write(str(i+1)+ ". "+doc["_source"]["product_description"])
    image = Image.open(temp_dir + doc["_source"]["image_url"])
    tab1.image(image)

# Multimodal search with both image and text caption as inputs
tab2.header("Multimodal search with both image and text caption as inputs")
embedding_model_id = tab2.text_input("Embedding model ID", placeholder="embedding_model_id", help="This is the embedding model you registered in the Jupyter notebook steps.")
query_2 = tab2.text_input("Keywords", placeholder="Type your search text here.")
query_2_image = tab2.file_uploader("Image", type=['png','jpeg','jpg'])

if not (embedding_model_id and query_2):
    st.stop()

#Multimodal Search
#Text as input
url_2 = 'https://'+aos_host+"/bedrock-multimodal-rag/_search"
text_embedding_payload_2 = {"_source": {
        "exclude": [
            "vector_embedding"
        ]
        },
        "query": {    
       
        "neural": {
            "vector_embedding": {
                
            #"query_image":query_image_binary,
            "query_text":query_2,
                
            "model_id": embedding_model_id,
            "k": 3
            }
            }
            
                    }
        
        ,"size":5,
  }

r_2 = requests.get(url_2, auth=awsauth, json=text_embedding_payload_2, headers=headers)
response_2 = json.loads(r_2.text)
docs_2 = response_2['hits']['hits']

for i,doc in enumerate(docs_2):
    tab2.write(str(i+1)+ ". "+doc["_source"]["product_description"])
    image = Image.open(temp_dir + doc["_source"]["image_url"])
    tab2.image(image)


### Run the code blocks below to install dependencies and launch the Streamlit app

In [ ]:
!pip install streamlit
!pip install opensearch-py
!pip install opensearch_py_ml
!pip install deprecated
!pip install requests_aws4auth
!pip install requests
!pip install PIL

Once the pip installs complete, run the following cell to launch the Streamlit application server.

In [ ]:
!streamlit run app.py --server.baseUrlPath="/proxy/absolute/8501"

[Application link](https://semantic-search-nb-qnru.notebook.us-east-1.sagemaker.aws/proxy/absolute/8501/)
https://semantic-search-nb-qnru.notebook.us-east-1.sagemaker.aws/proxy/absolute/8501/